In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.python.ops import bitwise_ops
from tensorflow.contrib import autograph

In [28]:
a = tf.placeholder(tf.float32, shape=[4, 128])    
b = tf.placeholder(tf.float32, shape=[128, 128])

In [36]:
def bitwise_xnor(a, b):
    # Need to do some dim expanding to handle batches.
    a = tf.expand_dims(a, axis=1)
    b = tf.expand_dims(b, axis=0)
    ab = bitwise_ops.invert(bitwise_ops.bitwise_xor(a, b))
    return ab

def binarize_dense(x, transpose=False):
    if transpose:
        x = tf.transpose(x, [1,0])
    h, w = x.shape
    num_bins = int(w / 64)
    binary_x = tf.cast(x > 0, tf.int64)
    packed_x= []
    for b in range(num_bins):
        packed_x.append(tf.zeros(shape=[h], dtype=tf.int64))
    for k in range(num_bins):
        for b in range(64):
            packed_x[k] = bitwise_ops.bitwise_or(packed_x[k], bitwise_ops.left_shift(binary_x[:, 64*k + b], b))
    packed_x = tf.stack(packed_x, axis=-1)     
    return packed_x

def binarize_dense_fast(x, transpose=False):
    if transpose:
        x = tf.transpose(x, [1,0])
    h, w = x.shape
    num_bins = int(w / 64)
    # Create shift tensor and apply it to binarized input.
    shift_bits = tf.broadcast_to(tf.range(64, dtype=tf.int64), x.shape)
    binary_x = tf.cast(x > 0, tf.int64)
    binary_x = bitwise_ops.left_shift(binary_x, shift_bits)
    # Split binarized x into chunks.
    binary_chunks = tf.split(binary_x, num_bins, axis=-1)
    # Combine chunks using bitwise or (equivalent to reduce sum).
    packed_x = tf.reduce_sum(binary_chunks, axis=-1)
    packed_x = tf.transpose(packed_x, [1,0])
    return packed_x
    
def binary_dense_matmul(a, b):
    ab = bitwise_xnor(a, b)
    pcnt = 2*(tf.cast(bitwise_ops.population_count(ab), tf.float32)) - 64
    inner_sum = tf.reduce_sum(pcnt, axis=-1)
    return inner_sum

def binary_dense(a, b, binarize_a=True, binarize_b=False):
    if binarize_a:
        bin_a = binarize_dense_fast(a)
    else:
        bin_a = a
    if binarize_b:
        bin_b = binarize_dense_fast(b, transpose=True)
    else:
        bin_b = tf.transpose(b, [1,0])
    return binary_dense_matmul(bin_a, bin_b)

In [37]:
bin_ab = binary_dense(a, b, binarize_b=True)
ab_reg = tf.matmul(tf.sign(a), tf.sign(b))

In [38]:
a_np = np.random.normal(size=a.shape.as_list())
b_np = np.random.normal(size=b.shape.as_list())

sess = tf.Session()
ab_out, bin_ab_out = sess.run([ab_reg, bin_ab], feed_dict={a:a_np, b:b_np})

In [39]:
bin_ab_out

array([[  0., -28., -12.,   0.,  18.,   8.,  -8., -14.,   8., -22.,   8.,
         -6.,   2.,  16., -16., -10.,   0., -12.,  -4.,   2., -10.,  -6.,
         -6.,   4., -16.,  -6.,   2.,  -6.,  18.,  -4.,  14.,  -4.,   0.,
          0.,  10.,  -2.,  18.,   0.,  -6.,  20.,  12.,  16.,  20., -24.,
          6.,   8.,   8.,   6.,  -2.,   4.,  -4.,  16.,   8.,   2.,   2.,
        -22.,  -2.,  10., -10., -10.,  -6.,   4.,  -8.,   0., -10.,  10.,
          0.,  -4.,  -6.,  34., -18., -10.,   0.,  20.,   8., -24.,  10.,
        -20.,  14.,  12.,  -4.,  -2.,   0., -18., -14.,  10.,  -4.,  12.,
          6.,   8.,  20., -16.,   0.,  16.,   6.,   8.,   0.,   6.,   4.,
        -16., -14.,   4., -16., -20.,  -4.,   2.,   0.,   6.,  -6.,   4.,
          2., -10.,  12.,  10.,  10., -12.,  -2.,   0.,  14.,  12.,   4.,
          2.,  14., -10.,   4., -10.,  26., -10.],
       [  0., -20.,  -8.,   0., -14.,   8.,  -4.,  10.,   0.,  -6.,   4.,
          6.,  10.,  -4.,   0.,  18.,   8.,   8.,  -8.,  -6.,

In [40]:
ab_out

array([[  0., -28., -12.,   0.,  18.,   8.,  -8., -14.,   8., -22.,   8.,
         -6.,   2.,  16., -16., -10.,   0., -12.,  -4.,   2., -10.,  -6.,
         -6.,   4., -16.,  -6.,   2.,  -6.,  18.,  -4.,  14.,  -4.,   0.,
          0.,  10.,  -2.,  18.,   0.,  -6.,  20.,  12.,  16.,  20., -24.,
          6.,   8.,   8.,   6.,  -2.,   4.,  -4.,  16.,   8.,   2.,   2.,
        -22.,  -2.,  10., -10., -10.,  -6.,   4.,  -8.,   0., -10.,  10.,
          0.,  -4.,  -6.,  34., -18., -10.,   0.,  20.,   8., -24.,  10.,
        -20.,  14.,  12.,  -4.,  -2.,   0., -18., -14.,  10.,  -4.,  12.,
          6.,   8.,  20., -16.,   0.,  16.,   6.,   8.,   0.,   6.,   4.,
        -16., -14.,   4., -16., -20.,  -4.,   2.,   0.,   6.,  -6.,   4.,
          2., -10.,  12.,  10.,  10., -12.,  -2.,   0.,  14.,  12.,   4.,
          2.,  14., -10.,   4., -10.,  26., -10.],
       [  0., -20.,  -8.,   0., -14.,   8.,  -4.,  10.,   0.,  -6.,   4.,
          6.,  10.,  -4.,   0.,  18.,   8.,   8.,  -8.,  -6.,